In [ ]:
import os
import ipykernel
import pathlib
from langchain import PromptTemplate, LLMChain
from pprint import pprint
import sys, os
current_working_directory = os.getcwd()
sys.path.append(os.path.dirname(current_working_directory))
nb_path = pathlib.Path(os.path.abspath(ipykernel.__file__)).parent.resolve(strict=True)
from models.glm_llm import glm_llm as glm
# llm = glm()


In [ ]:
os.environ["OPENAI_API_BASE"] = "https://u152649-b607-e934aa4f.westa.seetacloud.com:8443/v1"
os.environ["OPENAI_API_KEY"] = "EMPTY"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="chatglm2-6b", max_tokens=8192)

In [ ]:


question = "AI知识库用到的开源软件有哪些"
context = "以OpenAI公司GPT4为代表的⼤语⾔模型（LLM）涌现出很多强大的能力，利用LLM搭建具有语义检索和知识总结的新⼀代知识库成为了可能。LLM在铁路企业落地应⽤⾸先要解决⼤模型如何在本地部署的问题。今年以来，随着60亿650亿参数的LLaMA开源基础语⾔模型的快速发展，针对这类可以负担得起的大模型进行微调来完成企业特定任务成为新趋势。AI知识库⽤到的开源软件包括向量数据库和⼤模型两部分。向量数据库主要解决对⽂档进⾏语义检索，然后利⽤LLM对检索出来的结果进⾏提炼和总结并输出。我们选⽤的Milvus向量数据库和ChatGLM-6B⼤语⾔模型都是国内⽐较出⾊的开源软件，对中文语料支持较好。ChatGLM-6B对算⼒有⼀定要求，最好是支持Tensor Core的高性能显卡，在消费级显卡中，英伟达的RTX4090有24G显存，能满足Transformer模型的最低要求。目前京东上的公开报价约1.6w⼀张。在现阶段我们建议配置⼀台双RTX4090的工作站"
prompt_template = """基于以下已知信息，简洁和专业的来回答用户的问题。
                    如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
                    已知内容:
                    {context}
                    问题:
                    {question}"""
prompt = PromptTemplate(template=prompt_template,input_variables=["context", "question"])

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)
answer = llm_chain.predict(context = context, question = question)
print(answer)

In [ ]:
from langchain.base_language import BaseLanguageModel
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chains.summarize import map_reduce_prompt, refine_prompts, stuff_prompt, _load_map_reduce_chain, _load_refine_chain, _load_stuff_chain, LoadingCallable
from langchain.prompts.base import BasePromptTemplate
from typing import Optional, Any, Mapping
from prompts import stuff_prompt, refine_prompt, mapreduce_prompt
from loader.pdf_loader import loadDocsFromPDF
from loader.pdf_loader import loadDocsFromPDF2
from loader.chinese_text_splitter import ChineseProvisionsSplitter

In [ ]:
from pathlib import Path
# file_path = "/home/llp/llm/chatdoc-server/content/深圳站深圳车站旅客运输管理实施细则.pdf"
file_path = "/Users/llp/llm/chatdoc-server/content/客规/铁运〔1997〕101号-铁路旅客运输规程.pdf"
textsplitter = ChineseProvisionsSplitter(pdf=True)
docs = loadDocsFromPDF2(textsplitter, file_path,chunk_size = 300, chunk_overlap = 20)
# docs = loadDocsFromPDF(file_path,chunk_size = 300, chunk_overlap = 20)

pprint(f"docs size = {len(docs)}")
for doc in docs :
    page_numbers = doc.metadata["page_numbers"]
    content = doc.page_content
    print(f"#### content length= {len(doc.page_content)}    content=\n{content}")  

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """对以下内容写一个不少于500字的摘要:


"{text}"


摘要:"""
STUFF_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
summary_chain = load_summarize_chain(llm, chain_type="stuff", prompt = STUFF_PROMPT)
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)
# print(summarize_document_chain.run(docs[0].page_content))
# print('\n\n')
pprint(summary_chain.run(docs[:10]))


In [ ]:
prompt_template = """对以下内容写一个简洁的摘要:


{text}


摘要:"""
refine_summ_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

refine_template = (
    "你的工作是产生一个最终的摘要\n"
    "我们已经在一定程度上提供了一个摘要: {existing_answer}\n"
    "我们有机会通过下面一些更多的上下文来完善现有的摘要（仅在必要时）.\n"
   "------------\n"
    "{text}\n"
    "------------\n"
    "根据新的上下文完善现有的摘要"
    "如果上下文没有用处，请返回原始摘要。."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
summary_chain = load_summarize_chain(llm, chain_type="refine", question_prompt = refine_summ_prompt, refine_prompt = refine_prompt)
# summarize = summary_chain.run(docs[:3])
result = summary_chain(docs[:5])
pprint(result)
# print(f"【AI 回答】{result['output_text']}")



In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """对以下内容写一个简洁的摘要:


"{text}"


摘要:"""
STUFF_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])



In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
summary_chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt = STUFF_PROMPT, combine_prompt = STUFF_PROMPT, return_intermediate_steps=True)
summarize = summary_chain(docs[:5])
pprint(summarize)
# result = summary_chain({"input_documents": docs[:1]}, return_only_outputs=True)
# print(result["output_text"])



In [ ]:
from chains.qa_map_reduce_prompt import COMBINE_PROMPT, EXAMPLE_PROMPT,QUESTION_PROMPT
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
chain = load_qa_with_sources_chain(llm, chain_type="refine", question_prompt = QUESTION_PROMPT, combine_prompt = COMBINE_PROMPT, document_prompt = EXAMPLE_PROMPT)
query = "实名制售票的有关规定有哪些"
chain({"input_documents": docs[:5], "question": query}, return_only_outputs=True)


In [ ]:
from chains.qa_refine_prompt import DEFAULT_REFINE_PROMPT, DEFAULT_TEXT_QA_PROMPT, EXAMPLE_PROMPT
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
chain = load_qa_with_sources_chain(llm, chain_type="refine", question_prompt = DEFAULT_TEXT_QA_PROMPT, refine_prompt = DEFAULT_REFINE_PROMPT, document_prompt = EXAMPLE_PROMPT)
query = "车票票面应当载明什么？"
result = chain({"input_documents": docs[:2], "question": query}, return_only_outputs=False)
print(result)
print(f"【AI 回答】{result['output_text']}")


In [ ]:
import json
# from kb_select_prompt import KB_SELECT_PROMPT_TMPL
KB_SELECT_PROMPT_TMPL = """请从下面的文本中提取关键信息,并用json格式输出,我先给你几个例子,你将严格按照例子里的json格式输出,不要添加任何其他内容
问题：请从建设部的文件中查找关于建设项目资金管理方面的内容
回答："department':"建设部", "search_query":"建设资金管理", "知识库":"建设部知识库"
====================
问题：请检索建设部下发的文件，查找关于建设资金管理的信息
回答："department":"建设部","search_query":"建设项目资金管理", "知识库":"建设部知识库"
====================
问题：请打开建设部知识库，查找关于建设资金管理的信息
回答："department":"建设部", "search_query":"建设项目资金管理", "知识库":"建设部知识库"
====================
问题：请打开建设部知识库
回答："department":"建设部", "search_query": "", "知识库":"建设部知识库"
====================
问题：在建设部的文件中有哪些关于建设资金管理的内容
回答："department":"建设部", "search_query":"建设项目资金管理", "知识库":"建设部知识库"
====================
问题：请从客运部文件中查找安全生产的资料
回答："department":"客运部", "search_query":"安全生产", "知识库":"客运部知识库"
现在请从下面的文本中提取关键信息：
{question}
"""
question = "请从广铁集团客运部下发的管理文件中查找关于安全生产的资料"
prompt = PromptTemplate(template=KB_SELECT_PROMPT_TMPL,input_variables=["question"])
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)
answer = llm_chain.predict(question = question)
# 查找并去除"回答："
string = answer.replace("回答：", "")

# 将字符串周围的引号去掉，使其符合 JSON 格式
string = "{" + string.strip() + "}"
# 将双引号替换为单引号
string = string.replace("'", '"')
print(string)

# 将字符串转换为字典
dictionary = json.loads(string)

print(dictionary["知识库"])

In [14]:
os.environ["OPENAI_API_BASE"] = "https://u152649-b607-e934aa4f.westa.seetacloud.com:8443/v1"
os.environ["OPENAI_API_KEY"] = "EMPTY"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="chatglm2-6b", temperature=0.1, max_tokens=8192)
prompt_template = '''你是一个高度智能且能生成复杂问题-答案对的模型。
                    你以段落作为输入，并通过模仿人类提问和回答的方式，从输入的段落中生成尽可能详细，并且高质量、多样化的问题-答案对来进行总结。
                    要求你生成的问题-答案对数量不能少于5对。
                    你将严格从输入的段落中提取信息，决不会生成任何段落中没有的信息。
                    您的输出格式只能是【'Q': Question, 'A': Answer】形式，不能使用其他形式。
                    """
                    输入的段落:
                    {passage}
                    """
                    '''

# passage = "以OpenAI公司GPT4为代表的⼤语⾔模型（LLM）涌现出很多强大的能力，利用LLM搭建具有语义检索和知识总结的新⼀代知识库成为了可能。LLM在铁路企业落地应⽤⾸先要解决⼤模型如何在本地部署的问题。今年以来，诸多以LLaMA为基础的开源大语⾔模型的快速发展，针对这类可以负担得起的大模型进行微调来完成企业特定任务成为新趋势。AI知识库⽤到的开源软件包括向量数据库和⼤模型两部分。向量数据库主要解决对⽂档进⾏语义检索，然后利⽤LLM对检索出来的结果进⾏提炼和总结并输出。我们选⽤的Milvus向量数据库和国产ChatGLM6B⼤语⾔模型都是国内⽐较出⾊的开源软件，对中文语料支持较好。这些模型对算⼒有⼀定要求，最好是支持Tensor Core的高性能显卡，在消费级显卡中，英伟达的RTX4090有24G显存，能满足Transformer模型的最低要求。目前京东上的公开报价约1.6w⼀张。在现阶段我们建议配置⼀台双RTX4090的工作站"
# passage = "关于重新公布《铁路车号自动识别系统  AEI 设备管理检修运行规程》的通知    各铁路局：  铁路车号自动识别系统（ATIS）是铁路运输管理系统的重要组成部分，是铁路信息化的重要基础设施。车号地面识别设备（以下简称 AEI 设备）是车号系统的重要组成部分，是自动识别列车车辆，实现列车、机车、车辆实时跟踪的关键设备。为适应铁路运输发展和技术进步，全面加强 AEI 设备的检修、维护和管理工作，保证 ATIS 系统正常运行，铁道部重新制定了《铁路车号自动识别系统 AEI 设备管理检修运行规程》现予公布（另发单行本）， 技术规章编号：TG/CL219-2009，自 2009 年 5 月 1 日起施行，原发《铁路车号自动识别系统 AEI 设备管理检修运行规程》（铁运 [2003]123 号）同时废止。为贯彻落实好本规程，特要求如下：   —2—  一、认真学习。各单位要组织职工认真学习，全面领会，融会贯通，熟练掌握规程的各项要求、标准，对一线职工和管理人员认真组织培训，进行“应知、应会”考试，落实持证上岗制度。  二、抓好落实。各单位要结合实际，制定实施细则，细化各项规定，明确要求，将规程的要求、标准落实到岗位作业指导书； 积极开展 AEI 设备无故障运行考核活动，建立高效的设备故障应急抢修机制。各铁路局要认真总结规程执行中出现的问题，并及时报运输局装备部。  三、加强领导。车号自动识别系统是铁路信息化工作重要的信息源点。各级领导必须高度重视，确保健全管理制度、完善运行机制，并经常深入现场，加大规程落实的检查、指导、考核力度。  四、协调动作。有关部门要加强配合，分工负责，落实责任； 要建立跨部门合作的沟通协调机制，特别要提高结合点维护工作的效率和质量，确保系统正常运行"
# passage = "本规程对 AEI 基本功能、设备技术条件、产品上道准入、 制造检修单位资质、设备检修维护管理制度、检修技术标准和要求、相关单位管理职责等进行了规定，是 AEI 设备检修、维护、 管理工作基本依据，相关单位要结合实际，制订实施细则，落实到岗位作业指导书。遇有本规程规定不明确时，须按照保证设备稳定、可靠运行的原则，由铁路局车辆处牵头，检修维护单位、  —8—  设备生产厂家参加，共同研究制订技术标准，重大问题须上报铁道部。  1.6 AEI 设备检修和维护工作须严格执行本规程技术标准， 各单位要按规定配齐检测器具和检修、试验设备，并积极开发、 使用先进的检修工艺装备，积极采用远程诊断、动态检测等先进维护手段。重要配件实行专项修、集中修和换件修，不断提高检修、维护水平。  1.7 AEI 设备管理、检修和运行管理工作涉及运输、供电、 通信、工务、车辆、信息等专业，相关部门和单位须密切配合， 联劳协作，认真贯彻落实本规程的各项规定，同时严格执行铁道部和国家相关规章和技术标准。   1.8  新线规划和既有线改造设计须符合本规程的有关规定。  1.9  合资铁路和地方铁路 AEI 检修维护管理参照本规程执行。  1.10  本规程自 2009 年 5 月 1 日实施，铁道部前发有关规程、 文件、电报及技术标准与本规程不一致时，均以本规程为准。  1.11  本规程由铁道部负责解释、修改。    第二章系统构成    2.1 车号系统构成  车号系统由车辆电子标签（TAG）、地面自动识别设备（AEI）、 数据信息集中管理设备（CPS）、列检复示设备、车辆段复示设备、 铁路局分界站统计复示设备、铁路局监控系统、铁路局车号信息查询子系统、铁道部车号信息查询子系统、铁道部车号信息管理中央数据库、标签编程管理系统及通信网络等构成。见图 2.1    —9—  铁路计算机网铁道部车号信息管理中央数据库铁路局监控系统 CPS监控中心站修 AEI设备车辆段（厂） AEI设备站修CPS 车辆段CPS 铁路局应用服务器 AEI监控中心铁路局车号信息查询子系统铁道部车号信息查询子系统车站CPS 列检复示设备铁路局（集团公司）交界口、 车站、专用线出入口AEI设备车辆段复示   图 2.1 车号系统构成示意图  2.2 车辆电子标签（TAG）  车辆电子标签是车号系统重要部件之一"
# passage = "铁路旅客运输规程铁运〔1997〕101 号第一章总则第一条为了维护铁路旅客运输的正常秩序，保护铁路旅客运输合同各方当事人的合法权益，依据《中华人民共和国铁路法》制定本规程。 第二条本规程适用于中华人民共和国境内的铁路旅客和行李、包裹公共运输。 第三条国家铁路营业站的营业范围和与国家铁路办理直通运输业务的其他铁路营业站的营业范围以《铁路客运运价里程表》为准。其启用、封闭和营业范围的变更由国务院铁路主管部门批准，在《铁路客货运输专刊》上公布。 第四条铁路车站有关营业处所应有相应的票价表、运价表、杂费表、时刻表和旅客须知等内容。遇有变动，须于实施前通告。未经通告不得实施。 第五条下列用语在本规程内的意义： 承运人：与旅客或托运人签有运输合同的铁路运输企业。铁路车站、列车及与运营有关人员在执行职务中的行为代表承运人。 旅客：持有铁路有效乘车凭证的人和同行的免费乘车儿童。根据铁路货物运输合同押运货物的人视为旅客。 托运人：委托承运人运输行李或小件货物并与其签有行李包裹运输合同的人。 收货人：凭有效领取凭证领收行李、包裹的人。 直达票：从发站至到站不需中转换乘的车票。 通票：从发站至到站需中转换乘的车票。 改签：旅客变更乘车日期、车次、席（铺）位时需办理的签证手续。 等级：同等距离以承运人提供的乘车条件不同确定。 动车组：指运行速度在 200 公里及以上的列车。 客运记录：指在旅客或行李、包裹运输过程中因特殊情况，承运人与旅客、托运人、收货人之间需记载某种事项或车站与列车之间办理业务交接的文字凭证。 时间：以北京时间为准，从零时起计算，实行 24 小时制。 以上、以下、以前、以后、以内、以外：均含本数。 第六条在不违反本规程原则的前提下，铁路运输企业可根据具体情况制订补充规定在本企业管辖范围内实行并报国务院铁路主管部门备案。 第二章旅客运输第一节铁路旅客运输合同第七条铁路旅客运输合同是明确承运人与旅客之间权利义务关系的协议。起运地承运人依据本规程订立的旅客运输合同对所涉及的承运人具有同等约束力。 铁路旅客运输合同的基本凭证是车票。 第八条铁路旅客运输合同从售出车票时起成立，至按 票面规定运输结束旅客出站时止，为合同履行完毕。旅客运输的运送期间自检票进站起至到站出站时止计算。 第九条旅客的基本权利和义务是： 权利： 1．依据车票票面记载的内容乘车； 2．要求承运人提供与车票等级相适应的服务并保障其旅行安全； 3．对运送期间发生的身体损害有权要求承运人赔偿； 4．对运送期间因承运人过错造成的随身携带物品损失有权要求承运人赔偿。 义务： 1．支付运输费用，当场核对票、款，妥善保管车票， 保持票面信息完整可识别； 2．遵守国家法令和铁路运输规章制度，听从铁路车站、 列车工作人员的引导，按照车站的引导标志进、出站； 3．爱护铁路设备、设施，维护公共秩序和运输安全； 4．对所造成铁路或者其他旅客的损失予以赔偿。 第十条承运人的基本权利和义务是： 权利： 1．依照规定收取运输费用； 2．要求旅客遵守国家法令和铁路规章制度，保证安全； 3．对损害他人利益和铁路设备、设施的行为有权制止、 消除危险和要求赔偿。 义务： 1．确保旅客运输安全正点； 2．为旅客提供良好的旅行环境和服务设施，不断提高服务质量，文明礼貌地为旅客服务； 3．对运送期间发生的旅客身体损害予以赔偿； 4．对运送期间因承运人过错造成的旅客随身携带物品损失予以赔偿。 第二节车票第十一条车票票面（特殊票种除外）主要应当载明： 1．发站和到站站名； 2．座别、卧别； 3．径路； 4．票价； 5．车次； 6．乘车日期； 7．有效期。 第十二条车票中包括客票和附加票两部分。客票部分为软座、硬座。附加票部分为加快票、卧铺票、空调票。 附加票是客票的补充部分，可以与客票合并发售，但除儿童外不能单独使用。 第十三条车票票价为旅客乘车日的适用票价。承运人调整票价时，已售出的车票不再补收或退还票价差额。 第三节售票与购票 第十四条车票应在承运人或销售代理人的售票处购买。在有运输能力的情况下，承运人或销售代理人应按购票人的要求发售车票。 承运人可以开办往返票、联程票（指在购票地能够买到换乘地或返回地带有席位、铺位号的车票）、定期、不定期、 储值、定额等多种售票业务，以便于购票人购票和使用。 第十五条发售软座客票时最远至本次列车终点站。旅客在乘车区间中，要求一段乘坐硬座车，一段乘坐软座车时， 全程发售硬座客票。乘坐软座时，另收软座区间的软、硬座票价差额。 动车组列车车票最远只发售至本次列车终点站。 第十六条旅客购买加快票必须有软座或硬座客票。发售加快票的到站，必须是所乘快车或特别快车的停车站。发售需要中转换车的加快票的中转站还必须是有同等级快车始发的车站。"
# passage= "第十一条 车票票面（特殊票种除外）主要应当载明： 1．发站和到站站名； 2．座别、卧别； 3．径路； 4．票价； 5．车次； 6．乘车日期； 7．有效期。 第十二条 车票中包括客票和附加票两部分。客票部分 为软座、硬座。附加票部分为加快票、卧铺票、空调票。 附加票是客票的补充部分，可以与客票合并发售，但除 儿童外不能单独使用。 第十三条 车票票价为旅客乘车日的适用票价。承运人 调整票价时，已售出的车票不再补收或退还票价差额。"
total_questions = 5

for d in docs:
    passage = d.page_content
    prompt = PromptTemplate(template=prompt_template,input_variables=["passage",])
    llm_chain = LLMChain(
        llm=llm,
        prompt=prompt
    )
    answer = llm_chain.predict(passage = passage)
    print(answer)

Question: 什么是《中华人民共和国铁路法》?
Answer: 《中华人民共和国铁路法》是一部关于铁路运输的法律，旨在维护铁路旅客运输的正常秩序，保护铁路旅客运输合同各方当事人的合法权益。
Question: 什么是《中华人民共和国铁路旅客运输规程》?
Answer: 《中华人民共和国铁路旅客运输规程》是中国铁路总公司颁布的规范铁路旅客运输行为和运输质量的法规。它规定了铁路旅客运输的范围、条件、程序和责任等，旨在保障旅客的合法权益，维护铁路运输的安全和正常秩序。
Question: 什么是《铁路客运运价里程表》？
Answer: 《铁路客运运价里程表》是一个国家铁路主管部门批准发布的铁路客运运价里程表，用于规定国家铁路营业站的营业范围以及与国家铁路办理直通运输业务的其他铁路营业站的营业范围。
Question: 什么是铁路车站的票价表、运价表、杂费表、时刻表和旅客须知？
Answer: 铁路车站的票价表、运价表、杂费表、时刻表和旅客须知是铁路车站营业处所应有的相应内容。
问题1： 什么是承运人？
A：承运人是指与旅客或托运人签有运输合同的铁路运输企业。

问题2： 铁路车站、列车及与运营有关人员在执行职务中的行为代表谁？
A：铁路车站、列车及与运营有关人员在执行职务中的行为代表承运人。

问题3： 旅客的定义是什么？
A：旅客是指持有铁路有效乘车凭证的人和同行的免费乘车儿童。

问题4： 什么是托运人？
A：托运人是指委托承运人运输行李或小件货物并与其签有行李包裹运输合同的人。

问题5： 直达票和通票的区别是什么？
A：直达票指从发站至到站不需中转换乘的车票，而通票则指从发站至到站需中转换乘的车票。

问题6： 改签是什么手续？
A：改签是指旅客变更乘车日期、车次、席（铺）位时需办理的签证手续。

问题7： 动车组和客运记录有什么区别？
A：动车组指运行速度在200公里及以上的列车，而客运记录指在旅客或行李、包裹运输过程中因特殊情况，承运人与旅客、托运人、收货人之间需记载某种事项或车站与列车之间办理业务交接的文字凭证。
Q: 什么是铁路运输企业？
A: 铁路运输企业是指经营铁路运输业务的企业，包括铁路运输公司、铁路车站、铁路运输辅助企业等。
Q: 什么是铁路旅客运输合同?
A: 铁路旅客运输合同是明确承运人与旅客之间权利义务关系的协议。
Question: 